<p style = "font-family:courier,arial,helvetica;font-size:300%;">
Shopee - Price Match Gaurentee
</p>

![](https://img.lovepik.com/original_origin_pic/18/05/29/d49c1a025873dc18ca5919263197a3cc.png_wh300.png)

<p style = "font-family:courier,arial,helvetica;font-size:300%;">
Recent Update :
<ol style="font-family:courier,arial,helvetica;font-size:200%"><li>I added Phash Similarity Checker With a metric which is fast :)</li></ol>
</p>

<p style = "font-family:courier,arial,helvetica;font-size:300%;">
Importing Packages</p>

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import logging
import matplotlib.pyplot as plt
import transformers
from transformers import *
from tokenizers import BertWordPieceTokenizer
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
import cudf, cuml, cupy
import gc
import numpy as np
from sklearn.preprocessing import normalize
from scipy.spatial.distance import hamming
from IPython.display import clear_output
import imagehash

<p style = "font-family:courier,arial,helvetica;font-size:300%;">
Importing Saved Tokenizer</p>

In [ ]:
tokenizer = BertTokenizer.from_pretrained('../input/bert-large-tokenizer')
fast_tokenizer = BertWordPieceTokenizer('../input/bert-large-tokenizer/vocab.txt', lowercase=True )
fast_tokenizer

<p style = "font-family:courier,arial,helvetica;font-size:300%;">
Importing Test Data</p>

In [ ]:
test=pd.read_csv('../input/shopee-product-matching/test.csv')

In [ ]:
test.head()

<p style = "font-family:courier,arial,helvetica;font-size:300%;">
Encoding The Data</p>

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=35000, maxlen=200):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding()
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

<p style = "font-family:courier,arial,helvetica;font-size:300%;">
Finding Best Results</p>

In [ ]:
embeddings = fast_encode(test['title'].values, fast_tokenizer, maxlen=200)
embeddings=normalize(embeddings)
preds = []
CHUNK = 1024*4
text_embeddings=cupy.array(embeddings)
print('Finding similar titles...')
CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    print('chunk',a,'to',b)
    
    # COSINE SIMILARITY DISTANCE
    cts = cupy.matmul( text_embeddings, text_embeddings[a:b].T).T
    
    for k in range(b-a):
        IDX = cupy.where(cts[k,]>0.995)[0]
        o = test.iloc[cupy.asnumpy(IDX)].posting_id.values
        preds.append(o)
del text_embeddings,embeddings
_=gc.collect()

In [ ]:
posting=test['posting_id'].values
ans=[]
for i in range(len(test)):
    s=posting[i]
    if len(preds[i])>1 :
        for j in preds[i]:
            if j!=posting[i]:
                s+=' '+j
    ans.append(s)
preds=[]
for i in ans:
    preds.append(i.split(' '))

<p style = "font-family:courier,arial,helvetica;font-size:300%;">
Best Results Using PHASH</p>

<p style = "font-family:courier,arial,helvetica;font-size:200%;">
How is this working?? 🤔🤔
</p>
<p style = "font-family:courier,arial,helvetica;font-size:100%;">
    Well i had a basic idea of how i can filter True-False or False-True from the arrays so i used the XNOR function like this and took out the sum of them and subtracted them from 64 which is the highest value which made a distance . So if is closer to 0 means that the hash code has high similarity from the choosen hash code otherwise it is different from it :)
    <br>
    It was better than the time it took for hamming distance to run and seems to be good :)
</p>

In [ ]:
hash_codes=np.array([imagehash.hex_to_flathash(i,64).hash for i in test['image_phash'].values])
hash_preds=[]
count=1
for i in range(len(hash_codes)) :
    print(count*100/len(hash_codes),' % is done')
    clear_output(wait=True)
    count+=1
    x=sum((hash_codes[i]*hash_codes).T)
    y=sum((~hash_codes[i]*~hash_codes).T)
    t=x+y
    t=64-t
    IDX = np.where(t<10)[1]
    o = test.iloc[IDX].posting_id.values
    hash_preds.append(o)
hash_preds=[list(i) for i in hash_preds]

In [ ]:
hash_preds

<p style = "font-family:courier,arial,helvetica;font-size:300%;">
Result of Both Tokenizer and BaseLine Model Combined</p>

In [ ]:
%%time
test_list = test[['posting_id', 'title', 'image', 'image_phash']].reset_index(drop=True).values.tolist()
submission_list = []
# ==========================================
# Case. title[1] & image[2] & image_phash[3]
# ==========================================
count=0
for item in test_list:
    res1 = test[test['title']==item[1]]['posting_id'].unique().tolist()
    res2 = test[test['image']==item[2]]['posting_id'].unique().tolist()
    res2=list(set(res2+preds[count]))
    res3 = test[test['image_phash']==item[3]]['posting_id'].unique().tolist()
    res3=list(set(res3+hash_preds[count]))
    res = list(set(res1 + res2 + res3))
    res.remove(item[0])
    matches = ' '.join(map(str, res))
    matches = f'{item[0]} {matches}'
    submission_list.append([item[0], matches])
    count+=1



In [ ]:
submission = pd.DataFrame(submission_list, columns = ['posting_id','matches'])
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()

# Reference For Baseline model from : https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700 by chris deotte :)